# Lezione 21: Scelta del Numero di Cluster

## 🎯 Obiettivi della Lezione

Nella lezione precedente abbiamo sempre "saputo" quanti cluster cercare. Ma nella realtà, **K è sconosciuto**. Questa lezione affronta il problema centrale del clustering: **come scegliere K in modo sistematico**.

### Cosa Imparerai

| Metodo | Cosa Misura | Output |
|--------|-------------|--------|
| **Metodo del Gomito (Elbow)** | Inertia vs K | Punto di "ginocchio" |
| **Silhouette Analysis** | Qualità separazione | Score -1 to +1 |
| **Silhouette Plot** | Qualità per cluster | Visualizzazione dettagliata |
| **Gap Statistic** | Confronto con distribuzione random | K ottimale statistico |

### Prerequisiti
- ✅ Lezione 20: K-Means Clustering (centroidi, inertia, silhouette)
- ✅ Comprensione della distanza euclidea

### Outcome
Alla fine saprai:
1. Applicare 3+ metodi per scegliere K
2. Interpretare curve Elbow e Silhouette
3. Capire i trade-off tra i metodi
4. Evitare errori comuni nella scelta di K

---

```python
# Librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import make_blobs
```

---

## 1. Teoria: Il Problema della Scelta di K

### 1.1 Perché K è Così Importante?

La scelta di K determina completamente il risultato del clustering:

```
K troppo piccolo → Cluster eterogenei, informazione persa
K troppo grande  → Over-segmentazione, cluster senza significato
K giusto         → Gruppi naturali, interpretabili
```

**Il problema fondamentale:** K-Means funziona con QUALSIASI K, ma solo alcuni K rivelano la vera struttura dei dati.

### 1.2 L'Approccio Multi-Criterio

Non esiste un metodo perfetto. Ogni metodo ha pro e contro:

| Metodo | Pro | Contro |
|--------|-----|--------|
| **Elbow** | Intuitivo, veloce | "Gomito" spesso ambiguo |
| **Silhouette Score** | Misura oggettiva | Può favorire K piccoli |
| **Silhouette Plot** | Mostra problemi per cluster | Richiede interpretazione visiva |
| **Gap Statistic** | Fondamento statistico | Computazionalmente costoso |

**Best Practice:** Usa almeno 2 metodi e considera anche l'interpretabilità business.

---

### 1.3 Metodo del Gomito (Elbow Method)

**Idea:** L'inertia decresce sempre all'aumentare di K. Ma oltre un certo punto, il miglioramento diventa marginale.

**Formula Inertia:**
$$\text{Inertia} = \sum_{k=1}^{K} \sum_{i \in C_k} \|\mathbf{x}_i - \mathbf{c}_k\|^2$$

**Come funziona:**
1. Calcola l'inertia per K = 1, 2, 3, ..., K_max
2. Plotta la curva Inertia vs K
3. Cerca il "gomito" dove la curva piega

```
Inertia
   │
   │\
   │ \
   │  \____  ← Gomito: oltre questo K, poco miglioramento
   │       \____
   │            \____
   └───────────────────── K
   1   2   3   4   5   6
```

**Interpretazione:**
- Prima del gomito: K troppo piccolo, cluster eterogenei
- Al gomito: K ottimale (spesso)
- Dopo il gomito: Rendimenti decrescenti

---

### 1.4 Silhouette Score

**Idea:** Misura quanto bene ogni punto appartiene al suo cluster rispetto agli altri cluster.

**Formula per ogni punto i:**
$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

Dove:
- $a(i)$ = distanza media di i dai punti del **suo stesso cluster** (coesione)
- $b(i)$ = distanza media di i dai punti del **cluster più vicino** (separazione)

**Range:** da -1 a +1

| Valore | Significato |
|--------|-------------|
| **s ≈ +1** | Punto ben clusterizzato (lontano dagli altri cluster) |
| **s ≈ 0** | Punto al confine tra cluster |
| **s ≈ -1** | Punto probabilmente nel cluster sbagliato |

**Silhouette Score globale:** media di tutti i $s(i)$

**Interpretazione:**
- `> 0.7`: Eccellente separazione
- `0.5 - 0.7`: Buona struttura
- `0.25 - 0.5`: Struttura debole
- `< 0.25`: Cluster sovrapposti o struttura assente

---

### 1.5 Silhouette Plot (Analisi per Cluster)

Il silhouette score globale può nascondere problemi. Il **Silhouette Plot** mostra la distribuzione dei silhouette score per ogni cluster.

**Come leggerlo:**

```
Cluster 0  |████████████████|  → Cluster buono (tutti alti)
Cluster 1  |████████|          → Cluster piccolo ma buono
Cluster 2  |████|               → Cluster con valori bassi → PROBLEMA
           0       media      1
```

**Cosa cercare:**
1. **Larghezza uniforme:** tutti i cluster hanno dimensioni simili
2. **Nessun valore negativo:** nessun punto mal assegnato
3. **Valori sopra la media:** cluster di qualità

**Segnali di problemi:**
- Cluster con molti valori bassi o negativi
- Cluster molto sottili (pochi punti)
- Grande variabilità all'interno di un cluster

---

### 1.6 Gap Statistic

**Idea:** Confronta l'inertia del clustering reale con quella ottenuta su dati random uniformi.

**Intuizione:**
- Se i dati hanno vera struttura, l'inertia sarà molto più bassa rispetto ai dati random
- Il "gap" tra le due inertie indica la qualità del clustering

**Formula:**
$$\text{Gap}(K) = \mathbb{E}[\log(W_K^*)] - \log(W_K)$$

Dove:
- $W_K$ = inertia con K cluster sui dati reali
- $W_K^*$ = inertia con K cluster su dati random (media di B simulazioni)

**Scelta di K:** 
- Scegli il K più piccolo tale che: $\text{Gap}(K) \geq \text{Gap}(K+1) - s_{K+1}$
- Dove $s_{K+1}$ è la deviazione standard delle simulazioni

**Pro e Contro:**
- ✅ Base statistica rigorosa
- ✅ Funziona anche quando non c'è struttura (restituisce K=1)
- ❌ Computazionalmente costoso (molte simulazioni)
- ❌ Implementazione non in sklearn (serve codice custom)

---

## 2. Schema Mentale: Workflow per Scegliere K

```
                    DATI SCALATI
                         │
         ┌───────────────┼───────────────┐
         ▼               ▼               ▼
    ELBOW METHOD    SILHOUETTE      GAP STATISTIC
         │          ANALYSIS              │
         │               │                │
    Cerca il        Cerca il K        Criterio
    "gomito"        con score         statistico
                    massimo
         │               │                │
         └───────────────┴────────────────┘
                         │
                         ▼
              ┌──────────────────────┐
              │   I METODI CONCORDANO?│
              └──────────────────────┘
                    │           │
                   SI          NO
                    │           │
                    ▼           ▼
              Usa quel K    Considera:
                           • Interpretabilità
                           • Silhouette Plot
                           • Contesto business
                           • Prova più K
```

### Checklist Decisionale

1. ☐ Elbow Method → K candidato #1
2. ☐ Silhouette Score → K candidato #2  
3. ☐ Se concordano → quel K è probabilmente giusto
4. ☐ Se discordano → usa Silhouette Plot per decidere
5. ☐ Sempre verifica interpretabilità dei cluster
6. ☐ Mai fidarsi ciecamente di un solo metodo

---

## 3. Demo Pratiche

### Demo 1: Elbow Method con dati sintetici

Generiamo dati con 4 cluster ben separati e vediamo se l'Elbow Method trova K=4.

In [ ]:
# ============================================
# DEMO 1: Elbow Method
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import make_blobs

print("="*70)
print("DEMO 1: Elbow Method")
print("="*70)

# Generiamo dati con 4 cluster ben separati
np.random.seed(42)
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.8, random_state=42)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\n📊 Dataset: {X.shape[0]} punti, {X.shape[1]} feature")
print(f"🎯 Cluster veri: 4 (ma facciamo finta di non saperlo)")

# ============================================
# Calcolo Inertia per K = 1, 2, ..., 10
# ============================================
K_range = range(1, 11)
inertias = []

print("\n📈 Calcolo inertia per ogni K:")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    print(f"  K={k}: Inertia = {kmeans.inertia_:.2f}")

# ============================================
# Visualizzazione Elbow
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Curva Elbow
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Numero di Cluster (K)', fontsize=12)
axes[0].set_ylabel('Inertia (WCSS)', fontsize=12)
axes[0].set_title('Metodo del Gomito (Elbow Method)', fontsize=14)
axes[0].set_xticks(list(K_range))
axes[0].grid(True, alpha=0.3)

# Evidenziamo K=4
axes[0].axvline(x=4, color='red', linestyle='--', linewidth=2, label='K=4 (gomito)')
axes[0].scatter([4], [inertias[3]], color='red', s=200, zorder=5, marker='o')
axes[0].legend()

# Plot 2: Dati con cluster veri
scatter = axes[1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.7)
axes[1].set_xlabel('Feature 1 (scaled)')
axes[1].set_ylabel('Feature 2 (scaled)')
axes[1].set_title('Dati con cluster veri (K=4)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# Analisi del Gomito
# ============================================
print("\n" + "="*70)
print("📊 ANALISI DEL GOMITO")
print("="*70)

# Calcolo riduzione percentuale
print("\nRiduzione inertia passando da K a K+1:")
for i in range(1, len(inertias)):
    reduction = (inertias[i-1] - inertias[i]) / inertias[i-1] * 100
    marker = "← GOMITO!" if i == 4 else ""
    print(f"  K={i} → K={i+1}: -{reduction:.1f}% {marker}")

print("\n💡 INTERPRETAZIONE:")
print("   Il gomito è a K=4: dopo questo punto, aggiungere cluster")
print("   porta miglioramenti sempre più piccoli (rendimenti decrescenti)")
print("   Questo coincide con i 4 cluster che abbiamo generato!")

---

### Demo 2: Silhouette Score Analysis

Usiamo la silhouette score per confermare K=4 e vedere come varia con K diversi.

In [ ]:
# ============================================
# DEMO 2: Silhouette Score Analysis
# ============================================

print("="*70)
print("DEMO 2: Silhouette Score Analysis")
print("="*70)

# Usiamo gli stessi dati di prima (X_scaled)

# ============================================
# Calcolo Silhouette per K = 2, 3, ..., 10
# ============================================
K_range_sil = range(2, 11)  # Silhouette richiede almeno K=2
silhouette_scores = []

print("\n📈 Calcolo silhouette score per ogni K:")
for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)
    print(f"  K={k}: Silhouette = {score:.3f}")

# Trova il K migliore
best_k = K_range_sil[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)

print(f"\n🏆 MIGLIOR K secondo Silhouette: K={best_k} (score={best_score:.3f})")

# ============================================
# Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Silhouette vs K
axes[0].plot(list(K_range_sil), silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[0].set_xlabel('Numero di Cluster (K)', fontsize=12)
axes[0].set_ylabel('Silhouette Score', fontsize=12)
axes[0].set_title('Silhouette Score vs K', fontsize=14)
axes[0].set_xticks(list(K_range_sil))
axes[0].grid(True, alpha=0.3)

# Evidenziamo il K migliore
axes[0].axvline(x=best_k, color='red', linestyle='--', linewidth=2, label=f'K={best_k} (migliore)')
axes[0].scatter([best_k], [best_score], color='red', s=200, zorder=5, marker='*')
axes[0].legend()

# Linee di riferimento per interpretazione
axes[0].axhline(y=0.7, color='green', linestyle=':', alpha=0.7, label='Eccellente (>0.7)')
axes[0].axhline(y=0.5, color='orange', linestyle=':', alpha=0.7, label='Buono (>0.5)')
axes[0].axhline(y=0.25, color='red', linestyle=':', alpha=0.7, label='Debole (>0.25)')

# Plot 2: Confronto Elbow e Silhouette
ax2 = axes[1]
ax2_twin = ax2.twinx()

# Inertia (scala sinistra)
line1 = ax2.plot(list(K_range), inertias, 'b-o', linewidth=2, markersize=6, label='Inertia')
ax2.set_xlabel('Numero di Cluster (K)', fontsize=12)
ax2.set_ylabel('Inertia', color='blue', fontsize=12)
ax2.tick_params(axis='y', labelcolor='blue')

# Silhouette (scala destra)
line2 = ax2_twin.plot(list(K_range_sil), silhouette_scores, 'g-s', linewidth=2, markersize=6, label='Silhouette')
ax2_twin.set_ylabel('Silhouette Score', color='green', fontsize=12)
ax2_twin.tick_params(axis='y', labelcolor='green')

ax2.set_title('Confronto: Elbow vs Silhouette', fontsize=14)
ax2.set_xticks(list(K_range))
ax2.axvline(x=4, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# Interpretazione
# ============================================
print("\n" + "="*70)
print("📊 INTERPRETAZIONE")
print("="*70)
print(f"""
✅ ELBOW METHOD suggerisce: K=4 (gomito evidente)
✅ SILHOUETTE SCORE suggerisce: K={best_k} (score massimo)

I due metodi CONCORDANO! → Alta confidenza che K=4 è corretto.

📌 Nota: il silhouette score di {best_score:.3f} indica una struttura 
   {"eccellente" if best_score > 0.7 else "buona" if best_score > 0.5 else "moderata"} - i cluster sono ben separati.
""")

---

### Demo 3: Silhouette Plot (Analisi Dettagliata per Cluster)

Il silhouette plot mostra la distribuzione dei silhouette score per ogni cluster. È fondamentale per capire se ci sono cluster problematici.

In [ ]:
# ============================================
# DEMO 3: Silhouette Plot
# ============================================

print("="*70)
print("DEMO 3: Silhouette Plot per K=4")
print("="*70)

def plot_silhouette(X, n_clusters, ax, title):
    """Crea un silhouette plot per un dato K."""
    
    # Clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    
    # Silhouette per ogni punto
    silhouette_vals = silhouette_samples(X, labels)
    silhouette_avg = silhouette_score(X, labels)
    
    y_lower = 10
    colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))
    
    for i in range(n_clusters):
        # Valori silhouette per questo cluster, ordinati
        cluster_silhouette_vals = silhouette_vals[labels == i]
        cluster_silhouette_vals.sort()
        
        size_cluster_i = cluster_silhouette_vals.shape[0]
        y_upper = y_lower + size_cluster_i
        
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, cluster_silhouette_vals,
                          facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
        
        # Etichetta del cluster
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        
        y_lower = y_upper + 10
    
    ax.axvline(x=silhouette_avg, color="red", linestyle="--", 
               label=f'Media: {silhouette_avg:.3f}')
    ax.set_xlabel('Silhouette Score')
    ax.set_ylabel('Cluster')
    ax.set_title(title)
    ax.legend()
    ax.set_xlim([-0.1, 1])
    
    return labels, silhouette_avg

# ============================================
# Confronto K=3, K=4, K=5
# ============================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Riga superiore: Silhouette plots
for idx, k in enumerate([3, 4, 5]):
    labels, avg = plot_silhouette(X_scaled, k, axes[0, idx], f'Silhouette Plot: K={k}')

# Riga inferiore: Scatter plots
for idx, k in enumerate([3, 4, 5]):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    
    scatter = axes[1, idx].scatter(X_scaled[:, 0], X_scaled[:, 1], 
                                    c=labels, cmap='viridis', s=50, alpha=0.7)
    axes[1, idx].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                          c='red', marker='X', s=200, edgecolors='black', linewidths=2)
    axes[1, idx].set_xlabel('Feature 1')
    axes[1, idx].set_ylabel('Feature 2')
    axes[1, idx].set_title(f'K={k}, Silhouette={score:.3f}')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# Interpretazione
# ============================================
print("\n" + "="*70)
print("📊 COME LEGGERE IL SILHOUETTE PLOT")
print("="*70)
print("""
📌 COSA CERCARE:

1. LARGHEZZA UNIFORME
   - Tutti i cluster dovrebbero avere dimensioni simili
   - K=4: le 4 "lame" sono tutte circa uguali ✅
   - K=3: un cluster è molto più largo (unisce 2 cluster) ⚠️
   
2. VALORI SOPRA LA MEDIA
   - La linea rossa tratteggiata è la media globale
   - Buon cluster: la maggior parte dei punti è sopra la media
   - K=4: quasi tutti sopra la media ✅
   
3. NESSUN VALORE NEGATIVO
   - Valori negativi = punti nel cluster sbagliato
   - K=4: nessun valore negativo ✅
   
4. PUNTA DEL TRIANGOLO
   - Cluster ben formati hanno forma triangolare
   - Punte a destra = punti centrali, molto coesivi

💡 CONCLUSIONE: K=4 ha cluster uniformi, tutti sopra la media,
   nessun valore negativo → È la scelta migliore!
""")

---

### Demo 4: Gap Statistic (Implementazione Manuale)

Implementiamo la Gap Statistic per confrontarla con gli altri metodi.

In [ ]:
# ============================================
# DEMO 4: Gap Statistic
# ============================================

print("="*70)
print("DEMO 4: Gap Statistic")
print("="*70)

def compute_gap_statistic(X, k_max=10, n_refs=10):
    """
    Calcola la Gap Statistic per K = 1, 2, ..., k_max.
    
    Args:
        X: dati (già scalati)
        k_max: massimo numero di cluster da testare
        n_refs: numero di dataset random per ogni K
    
    Returns:
        gaps: lista dei gap values
        sk: lista delle deviazioni standard
    """
    
    # Range dei dati per generare dati uniformi
    mins = X.min(axis=0)
    maxs = X.max(axis=0)
    
    gaps = []
    sks = []
    
    for k in range(1, k_max + 1):
        # Inertia sui dati reali
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        log_Wk = np.log(kmeans.inertia_)
        
        # Inertia su dati random (n_refs simulazioni)
        ref_inertias = []
        for _ in range(n_refs):
            # Genera dati uniformi nello stesso range
            X_random = np.random.uniform(mins, maxs, size=X.shape)
            kmeans_ref = KMeans(n_clusters=k, random_state=None, n_init=10)
            kmeans_ref.fit(X_random)
            ref_inertias.append(np.log(kmeans_ref.inertia_))
        
        # Gap = E[log(W*_k)] - log(W_k)
        gap = np.mean(ref_inertias) - log_Wk
        gaps.append(gap)
        
        # Deviazione standard per il criterio di arresto
        sdk = np.std(ref_inertias) * np.sqrt(1 + 1/n_refs)
        sks.append(sdk)
    
    return gaps, sks

print("\n⏳ Calcolo Gap Statistic (potrebbe richiedere qualche secondo)...")
gaps, sks = compute_gap_statistic(X_scaled, k_max=8, n_refs=10)

print("\n📈 Risultati Gap Statistic:")
for k, (gap, sk) in enumerate(zip(gaps, sks), 1):
    print(f"  K={k}: Gap = {gap:.3f}, sd = {sk:.3f}")

# Trova K ottimale secondo il criterio: Gap(k) >= Gap(k+1) - s(k+1)
optimal_k = 1
for k in range(len(gaps) - 1):
    if gaps[k] >= gaps[k+1] - sks[k+1]:
        optimal_k = k + 1
        break

print(f"\n🏆 K ottimale secondo Gap Statistic: K={optimal_k}")

# ============================================
# Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

K_range_gap = range(1, len(gaps) + 1)

# Plot 1: Gap values con barre di errore
axes[0].errorbar(K_range_gap, gaps, yerr=sks, fmt='o-', capsize=5, 
                  linewidth=2, markersize=8, color='purple')
axes[0].axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, 
                 label=f'K ottimale = {optimal_k}')
axes[0].set_xlabel('Numero di Cluster (K)', fontsize=12)
axes[0].set_ylabel('Gap Statistic', fontsize=12)
axes[0].set_title('Gap Statistic vs K', fontsize=14)
axes[0].set_xticks(list(K_range_gap))
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Plot 2: Confronto tutti i metodi
# Normalizziamo per visualizzare insieme
inertias_norm = (np.array(inertias) - min(inertias)) / (max(inertias) - min(inertias))
inertias_norm = 1 - inertias_norm  # Invertiamo: più alto = meglio

silhouette_norm = (np.array(silhouette_scores) - min(silhouette_scores)) / (max(silhouette_scores) - min(silhouette_scores))

gaps_norm = (np.array(gaps) - min(gaps)) / (max(gaps) - min(gaps))

axes[1].plot(range(1, 11), inertias_norm, 'b-o', label='Elbow (invertito)', linewidth=2)
axes[1].plot(range(2, 11), silhouette_norm, 'g-s', label='Silhouette', linewidth=2)
axes[1].plot(range(1, 9), gaps_norm, 'm-^', label='Gap Statistic', linewidth=2)
axes[1].axvline(x=4, color='red', linestyle='--', linewidth=2, label='K=4')
axes[1].set_xlabel('Numero di Cluster (K)', fontsize=12)
axes[1].set_ylabel('Score (normalizzato)', fontsize=12)
axes[1].set_title('Confronto Tutti i Metodi (normalizzati)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(range(1, 11))

plt.tight_layout()
plt.show()

# ============================================
# Riepilogo
# ============================================
print("\n" + "="*70)
print("📊 RIEPILOGO: TUTTI I METODI CONCORDANO!")
print("="*70)
print(f"""
   Metodo              K Suggerito
   ─────────────────────────────────
   Elbow Method        K = 4
   Silhouette Score    K = 4
   Gap Statistic       K = {optimal_k}
   
   Cluster veri        K = 4 ✅
   
💡 Quando i metodi concordano, hai alta confidenza nella scelta!
""")

---

### Demo 5: Caso Ambiguo - Quando i Metodi Non Concordano

Nella realtà, i metodi spesso danno risultati diversi. Vediamo un caso più difficile.

In [ ]:
# ============================================
# DEMO 5: Caso Ambiguo
# ============================================

print("="*70)
print("DEMO 5: Caso Ambiguo - Cluster Sovrapposti")
print("="*70)

# Generiamo dati con cluster che si sovrappongono parzialmente
np.random.seed(42)
X_hard, _ = make_blobs(n_samples=300, centers=5, cluster_std=1.8, random_state=42)
X_hard_scaled = StandardScaler().fit_transform(X_hard)

print("\n📊 Dataset difficile: 5 cluster parzialmente sovrapposti")

# ============================================
# Elbow Method
# ============================================
K_range = range(1, 11)
inertias_hard = []
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_hard_scaled)
    inertias_hard.append(km.inertia_)

# ============================================
# Silhouette Scores
# ============================================
K_range_sil = range(2, 11)
sil_scores_hard = []
for k in K_range_sil:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_hard_scaled)
    sil_scores_hard.append(silhouette_score(X_hard_scaled, labels))

# ============================================
# Visualizzazione
# ============================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Plot 1: Elbow
axes[0, 0].plot(K_range, inertias_hard, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('K')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method - Gomito non chiaro!')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(list(K_range))

# Evidenziamo l'ambiguità
for k in [3, 4, 5]:
    axes[0, 0].axvline(x=k, color='orange', linestyle='--', alpha=0.5)
axes[0, 0].text(4, max(inertias_hard)*0.8, '?', fontsize=30, color='red', ha='center')

# Plot 2: Silhouette
axes[0, 1].plot(list(K_range_sil), sil_scores_hard, 'go-', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('K')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score - Variazione piccola')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(list(K_range_sil))

best_k_hard = list(K_range_sil)[np.argmax(sil_scores_hard)]
axes[0, 1].axvline(x=best_k_hard, color='red', linestyle='--', label=f'Max: K={best_k_hard}')
axes[0, 1].legend()

# Plot 3: Dati originali
axes[0, 2].scatter(X_hard_scaled[:, 0], X_hard_scaled[:, 1], s=30, alpha=0.5)
axes[0, 2].set_title('Dati originali')
axes[0, 2].grid(True, alpha=0.3)

# Plots inferiori: Silhouette plots per K=3, 4, 5
for idx, k in enumerate([3, 4, 5]):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_hard_scaled)
    score = silhouette_score(X_hard_scaled, labels)
    
    # Silhouette per ogni cluster
    silhouette_vals = silhouette_samples(X_hard_scaled, labels)
    
    y_lower = 10
    for i in range(k):
        cluster_vals = silhouette_vals[labels == i]
        cluster_vals.sort()
        size_cluster = cluster_vals.shape[0]
        y_upper = y_lower + size_cluster
        
        color = plt.cm.viridis(i / k)
        axes[1, idx].fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_vals,
                                    facecolor=color, edgecolor=color, alpha=0.7)
        y_lower = y_upper + 10
    
    axes[1, idx].axvline(x=score, color='red', linestyle='--')
    axes[1, idx].set_title(f'K={k}, Silhouette={score:.3f}')
    axes[1, idx].set_xlabel('Silhouette Score')
    axes[1, idx].set_xlim([-0.1, 1])

plt.tight_layout()
plt.show()

# ============================================
# Interpretazione
# ============================================
print("\n" + "="*70)
print("📊 ANALISI DEL CASO AMBIGUO")
print("="*70)
print(f"""
❌ ELBOW: Nessun gomito chiaro - la curva scende gradualmente
❓ SILHOUETTE: Poca differenza tra K=3, 4, 5 (tutti ~0.3-0.4)

🔍 Come decidere?

1. SILHOUETTE PLOT rivela:
   - K=3: cluster più uniformi, meno valori negativi
   - K=4, K=5: alcuni cluster sottili → forse over-segmentazione
   
2. CONSIDERA IL CONTESTO:
   - Se 3 gruppi sono sufficienti per il business → K=3
   - Se serve più granularità → K=4 o K=5
   
3. REGOLA PRATICA:
   Quando i metodi non concordano, scegli K più piccolo
   (più facile da interpretare, meno overfitting)

💡 LEZIONE: Non sempre esiste un K "giusto" - a volte la struttura
   è debole e bisogna accettare l'incertezza.
""")

---

## 4. Esercizi

### 📝 Esercizio 21.1 — Funzione Riutilizzabile per Elbow + Silhouette

**Consegna:**
Crea una funzione `analisi_k_ottimale(X, k_min, k_max)` che:
1. Calcola inertia e silhouette score per ogni K nel range
2. Crea un plot con 2 assi Y (Elbow e Silhouette insieme)
3. Restituisce il K con silhouette massima
4. Stampa una raccomandazione

**Test:** Applica la funzione ai dati `make_blobs(n_samples=200, centers=3, cluster_std=0.8)`

In [ ]:
# ============================================
# ESERCIZIO 21.1 — SOLUZIONE
# ============================================

def analisi_k_ottimale(X, k_min=2, k_max=10):
    """
    Analizza il numero ottimale di cluster usando Elbow e Silhouette.
    
    Args:
        X: dati (già scalati)
        k_min: K minimo da testare (default 2)
        k_max: K massimo da testare (default 10)
    
    Returns:
        k_ottimale: K con silhouette score massima
    """
    
    print("="*70)
    print("ANALISI K OTTIMALE - Elbow + Silhouette")
    print("="*70)
    
    K_range = range(k_min, k_max + 1)
    inertias = []
    silhouettes = []
    
    # Calcolo metriche per ogni K
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        
        inertias.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(X, labels))
    
    # K ottimale secondo silhouette
    k_ottimale = list(K_range)[np.argmax(silhouettes)]
    best_silhouette = max(silhouettes)
    
    # ============================================
    # Visualizzazione con doppio asse Y
    # ============================================
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Asse sinistro: Inertia (Elbow)
    color1 = 'tab:blue'
    ax1.set_xlabel('Numero di Cluster (K)', fontsize=12)
    ax1.set_ylabel('Inertia (WCSS)', color=color1, fontsize=12)
    line1 = ax1.plot(K_range, inertias, 'o-', color=color1, linewidth=2, 
                      markersize=8, label='Inertia (Elbow)')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, alpha=0.3)
    
    # Asse destro: Silhouette
    ax2 = ax1.twinx()
    color2 = 'tab:green'
    ax2.set_ylabel('Silhouette Score', color=color2, fontsize=12)
    line2 = ax2.plot(K_range, silhouettes, 's-', color=color2, linewidth=2, 
                      markersize=8, label='Silhouette')
    ax2.tick_params(axis='y', labelcolor=color2)
    
    # Evidenzia K ottimale
    ax1.axvline(x=k_ottimale, color='red', linestyle='--', linewidth=2, 
                 label=f'K ottimale = {k_ottimale}')
    
    # Legenda combinata
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='center right')
    
    plt.title(f'Elbow + Silhouette Analysis\nK ottimale: {k_ottimale} (Silhouette: {best_silhouette:.3f})', 
              fontsize=14)
    ax1.set_xticks(list(K_range))
    
    plt.tight_layout()
    plt.show()
    
    # ============================================
    # Report
    # ============================================
    print(f"\n📊 RISULTATI:")
    print(f"{'K':<5} {'Inertia':<12} {'Silhouette':<12}")
    print("-" * 30)
    for k, (inert, sil) in zip(K_range, zip(inertias, silhouettes)):
        marker = "← MIGLIORE" if k == k_ottimale else ""
        print(f"{k:<5} {inert:<12.2f} {sil:<12.3f} {marker}")
    
    print(f"\n🏆 RACCOMANDAZIONE: K = {k_ottimale}")
    print(f"   Silhouette Score: {best_silhouette:.3f}", end=" ")
    if best_silhouette > 0.7:
        print("(Eccellente separazione)")
    elif best_silhouette > 0.5:
        print("(Buona struttura)")
    elif best_silhouette > 0.25:
        print("(Struttura moderata)")
    else:
        print("(Struttura debole)")
    
    return k_ottimale

# ============================================
# TEST
# ============================================
print("\n" + "="*70)
print("TEST della funzione")
print("="*70)

# Generiamo dati di test
X_test, _ = make_blobs(n_samples=200, centers=3, cluster_std=0.8, random_state=42)
X_test_scaled = StandardScaler().fit_transform(X_test)

k_risultato = analisi_k_ottimale(X_test_scaled, k_min=2, k_max=8)

print(f"\n✅ La funzione ha identificato K = {k_risultato}")
print(f"   (I dati avevano 3 cluster reali)")

---

### 📝 Esercizio 21.2 — Silhouette Plot Dettagliato

**Consegna:**
Crea una funzione `visualizza_silhouette_clusters(X, k)` che:
1. Applica K-Means con K cluster
2. Crea un silhouette plot con:
   - Ogni cluster colorato diversamente
   - La media globale evidenziata
   - Statistiche per cluster (n punti, media silhouette)
3. Identifica cluster "problematici" (silhouette media < 0.25)

**Test:** Applica a dati con cluster di dimensioni diverse.

In [ ]:
# ============================================
# ESERCIZIO 21.2 — SOLUZIONE
# ============================================

def visualizza_silhouette_clusters(X, k):
    """
    Crea un silhouette plot dettagliato con statistiche per cluster.
    
    Args:
        X: dati (già scalati)
        k: numero di cluster
    
    Returns:
        cluster_stats: dict con statistiche per cluster
    """
    
    print("="*70)
    print(f"SILHOUETTE PLOT DETTAGLIATO - K={k}")
    print("="*70)
    
    # Clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    
    # Silhouette per ogni punto
    silhouette_vals = silhouette_samples(X, labels)
    silhouette_avg = silhouette_score(X, labels)
    
    # ============================================
    # Statistiche per cluster
    # ============================================
    cluster_stats = {}
    print(f"\n📊 Statistiche per cluster:")
    print(f"{'Cluster':<10} {'N punti':<10} {'Sil Media':<12} {'Sil Min':<10} {'Status':<15}")
    print("-" * 60)
    
    for i in range(k):
        cluster_vals = silhouette_vals[labels == i]
        stats = {
            'n_punti': len(cluster_vals),
            'sil_mean': cluster_vals.mean(),
            'sil_min': cluster_vals.min(),
            'sil_max': cluster_vals.max(),
            'problematico': cluster_vals.mean() < 0.25
        }
        cluster_stats[i] = stats
        
        status = "⚠️ PROBLEMATICO" if stats['problematico'] else "✅ OK"
        print(f"{i:<10} {stats['n_punti']:<10} {stats['sil_mean']:<12.3f} {stats['sil_min']:<10.3f} {status:<15}")
    
    print(f"\n📈 Silhouette media globale: {silhouette_avg:.3f}")
    
    # ============================================
    # Visualizzazione
    # ============================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Colori per cluster
    colors = plt.cm.viridis(np.linspace(0, 1, k))
    
    # Plot 1: Silhouette Plot
    y_lower = 10
    for i in range(k):
        cluster_vals = silhouette_vals[labels == i]
        cluster_vals.sort()
        
        size_cluster = cluster_vals.shape[0]
        y_upper = y_lower + size_cluster
        
        color = colors[i]
        axes[0].fill_betweenx(np.arange(y_lower, y_upper),
                               0, cluster_vals,
                               facecolor=color, edgecolor=color, alpha=0.7)
        
        # Etichetta con statistiche
        sil_mean = cluster_vals.mean()
        axes[0].text(-0.05, y_lower + 0.5 * size_cluster, 
                      f'{i}: μ={sil_mean:.2f}', fontsize=9)
        
        y_lower = y_upper + 10
    
    # Linea della media globale
    axes[0].axvline(x=silhouette_avg, color='red', linestyle='--', 
                     linewidth=2, label=f'Media: {silhouette_avg:.3f}')
    
    # Soglia problematica
    axes[0].axvline(x=0.25, color='orange', linestyle=':', 
                     linewidth=2, alpha=0.7, label='Soglia (0.25)')
    axes[0].axvline(x=0, color='black', linestyle='-', linewidth=1)
    
    axes[0].set_xlabel('Silhouette Score', fontsize=12)
    axes[0].set_ylabel('Cluster', fontsize=12)
    axes[0].set_title(f'Silhouette Plot (K={k})', fontsize=14)
    axes[0].set_xlim([-0.2, 1])
    axes[0].legend(loc='lower right')
    
    # Plot 2: Scatter con cluster
    scatter = axes[1].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', 
                               s=50, alpha=0.7)
    axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                     c='red', marker='X', s=200, edgecolors='black', linewidths=2,
                     label='Centroidi')
    axes[1].set_xlabel('Feature 1', fontsize=12)
    axes[1].set_ylabel('Feature 2', fontsize=12)
    axes[1].set_title(f'Cluster (Silhouette={silhouette_avg:.3f})', fontsize=14)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # ============================================
    # Identificazione cluster problematici
    # ============================================
    problematici = [i for i, s in cluster_stats.items() if s['problematico']]
    if problematici:
        print(f"\n⚠️ CLUSTER PROBLEMATICI: {problematici}")
        print("   Questi cluster hanno silhouette media < 0.25")
        print("   Possibili cause: sovrapposizione, dimensione piccola, outlier")
    else:
        print("\n✅ Tutti i cluster sono ben formati!")
    
    return cluster_stats

# ============================================
# TEST
# ============================================
print("\n" + "="*70)
print("TEST con cluster di dimensioni diverse")
print("="*70)

# Generiamo cluster di dimensioni molto diverse
np.random.seed(42)
cluster1 = np.random.randn(150, 2) + np.array([0, 0])      # Grande
cluster2 = np.random.randn(30, 2) * 0.5 + np.array([5, 5])  # Piccolo, compatto
cluster3 = np.random.randn(70, 2) * 1.2 + np.array([2, 5])  # Medio, sparso

X_unbalanced = np.vstack([cluster1, cluster2, cluster3])
X_unbalanced_scaled = StandardScaler().fit_transform(X_unbalanced)

stats = visualizza_silhouette_clusters(X_unbalanced_scaled, k=3)

---

### 📝 Esercizio 21.3 — Analisi Completa su Dataset Reale

**Consegna:**
Hai dati di un sondaggio con 3 variabili numeriche. Determina il numero ottimale di segmenti di clienti usando:
1. Elbow Method
2. Silhouette Score
3. Silhouette Plot per i K candidati
4. Decisione finale motivata

**Dataset:**
```python
soddisfazione = [8,7,9,3,2,4,6,5,7,9,8,2,3,4,6,7,8,9,1,2]
frequenza_acquisto = [5,4,6,1,2,1,3,3,4,5,6,1,1,2,3,4,5,6,1,1]
spesa_mensile = [200,180,250,50,40,60,100,80,150,220,200,30,45,55,90,160,190,230,25,35]
```

In [ ]:
# ============================================
# ESERCIZIO 21.3 — SOLUZIONE
# ============================================

import pandas as pd

print("="*70)
print("ESERCIZIO 21.3 — Analisi Completa Segmentazione Clienti")
print("="*70)

# ============================================
# PASSO 1: Preparazione dati
# ============================================
soddisfazione = [8,7,9,3,2,4,6,5,7,9,8,2,3,4,6,7,8,9,1,2]
frequenza_acquisto = [5,4,6,1,2,1,3,3,4,5,6,1,1,2,3,4,5,6,1,1]
spesa_mensile = [200,180,250,50,40,60,100,80,150,220,200,30,45,55,90,160,190,230,25,35]

df_clienti = pd.DataFrame({
    'soddisfazione': soddisfazione,
    'frequenza': frequenza_acquisto,
    'spesa': spesa_mensile
})

print("\n📊 Dataset:")
print(df_clienti.describe().round(1))

# Scaling
X_clienti = df_clienti.values
scaler = StandardScaler()
X_clienti_scaled = scaler.fit_transform(X_clienti)

# ============================================
# PASSO 2: Elbow Method
# ============================================
print("\n" + "="*70)
print("PASSO 2: Elbow Method")
print("="*70)

K_range = range(2, 8)
inertias = []
silhouettes = []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_clienti_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_clienti_scaled, labels))
    print(f"  K={k}: Inertia={km.inertia_:.2f}, Silhouette={silhouettes[-1]:.3f}")

# ============================================
# PASSO 3: Visualizzazione Elbow + Silhouette
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('K')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Metodo del Gomito')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(list(K_range))

# Silhouette
axes[1].plot(list(K_range), silhouettes, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('K')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(list(K_range))

best_k_sil = list(K_range)[np.argmax(silhouettes)]
axes[1].axvline(x=best_k_sil, color='red', linestyle='--', label=f'Migliore: K={best_k_sil}')
axes[1].legend()

plt.tight_layout()
plt.show()

# ============================================
# PASSO 4: Silhouette Plot per K candidati
# ============================================
print("\n" + "="*70)
print("PASSO 4: Silhouette Plot per K=2, 3, 4")
print("="*70)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, k in enumerate([2, 3, 4]):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_clienti_scaled)
    sil_vals = silhouette_samples(X_clienti_scaled, labels)
    sil_avg = silhouette_score(X_clienti_scaled, labels)
    
    y_lower = 10
    colors = plt.cm.viridis(np.linspace(0, 1, k))
    
    for i in range(k):
        cluster_vals = sil_vals[labels == i]
        cluster_vals.sort()
        size = len(cluster_vals)
        y_upper = y_lower + size
        
        axes[idx].fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_vals,
                                 facecolor=colors[i], alpha=0.7)
        axes[idx].text(-0.05, y_lower + size/2, f'C{i}', fontsize=10)
        y_lower = y_upper + 10
    
    axes[idx].axvline(x=sil_avg, color='red', linestyle='--', linewidth=2)
    axes[idx].axvline(x=0, color='black', linestyle='-', linewidth=1)
    axes[idx].set_xlim([-0.2, 1])
    axes[idx].set_xlabel('Silhouette Score')
    axes[idx].set_title(f'K={k} (avg={sil_avg:.3f})')

plt.tight_layout()
plt.show()

# ============================================
# PASSO 5: Decisione Finale
# ============================================
print("\n" + "="*70)
print("PASSO 5: DECISIONE FINALE")
print("="*70)

print(f"""
📊 ANALISI:

1. ELBOW METHOD:
   - Gomito non chiarissimo, ma suggerisce K=2 o K=3
   
2. SILHOUETTE SCORE:
   - K=2: {silhouettes[0]:.3f}
   - K=3: {silhouettes[1]:.3f}
   - K=4: {silhouettes[2]:.3f}
   - Migliore: K={best_k_sil}
   
3. SILHOUETTE PLOT:
   - K=2: cluster uniformi, buona separazione
   - K=3: cluster ancora uniformi
   - K=4: cluster più piccoli, possibile over-segmentazione

🏆 DECISIONE: K = {best_k_sil}

MOTIVAZIONE:
- Silhouette score massima a K={best_k_sil}
- I cluster sono ben bilanciati nel silhouette plot
- {best_k_sil} segmenti sono interpretabili per il marketing
""")

# ============================================
# Visualizzazione Cluster Finali
# ============================================
km_finale = KMeans(n_clusters=best_k_sil, random_state=42, n_init=10)
df_clienti['cluster'] = km_finale.fit_predict(X_clienti_scaled)

print("\n📌 PROFILI DEI CLUSTER:")
print(df_clienti.groupby('cluster').mean().round(1))

---

## 5. Cosa Portarsi a Casa

### ✅ Concetti Fondamentali

| Metodo | Cosa Misura | Come Scegliere K |
|--------|-------------|------------------|
| **Elbow** | Inertia | Punto di "gomito" nella curva |
| **Silhouette Score** | Qualità separazione | K con score massimo |
| **Silhouette Plot** | Distribuzione per cluster | Cluster uniformi, sopra media |
| **Gap Statistic** | Confronto con random | Criterio statistico |

### ⚠️ Errori Comuni da Evitare

| Errore | Problema | Soluzione |
|--------|----------|-----------|
| Usare solo un metodo | Risultato non affidabile | Sempre almeno 2 metodi |
| Ignorare il silhouette plot | Non vedi cluster problematici | Sempre analisi visiva |
| Scegliere K solo per metrica | Cluster non interpretabili | Considera il business |
| Aspettarsi sempre un K chiaro | Frustrazione | A volte la struttura è debole |

### 🔗 Ponte verso la Lezione 22

**Problema:** K-Means richiede K a priori e assume cluster sferici.

**E se non sapessimo nulla sulla forma dei cluster?**

Nella prossima lezione:
- **Clustering Gerarchico**: non richiede K a priori
- **Dendrogrammi**: visualizzano la struttura gerarchica
- **Metodi di linkage**: single, complete, average, Ward
- **Taglio del dendrogramma**: come scegliere K a posteriori

---

## 📋 BIGNAMI — Lezione 21: Scelta del Numero di Cluster

### Definizioni Essenziali

| Termine | Definizione |
|---------|-------------|
| **Elbow Method** | Tecnica che cerca il "gomito" nella curva Inertia vs K |
| **Silhouette Score** | Media di $s(i)$ su tutti i punti; misura qualità clustering |
| **Silhouette Plot** | Visualizzazione della distribuzione di $s(i)$ per cluster |
| **Gap Statistic** | Differenza tra log-inertia reale e attesa su dati random |
| **WCSS (Inertia)** | Within-Cluster Sum of Squares - somma distanze quadrate |

---

### Formule Chiave

**Silhouette per punto i:**
$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

- $a(i)$: distanza media intra-cluster (coesione)
- $b(i)$: distanza media al cluster più vicino (separazione)

**Interpretazione Silhouette:**
| Range | Qualità |
|-------|---------|
| 0.7 - 1.0 | Eccellente |
| 0.5 - 0.7 | Buona |
| 0.25 - 0.5 | Moderata |
| < 0.25 | Debole/Problematica |

---

### Checklist per Scegliere K

```
□ Applica StandardScaler ai dati
□ Calcola Elbow (Inertia vs K) → Individua gomito
□ Calcola Silhouette Score vs K → Trova massimo
□ I metodi concordano? → Usa quel K
□ I metodi discordano? → Analizza Silhouette Plot
□ Verifica interpretabilità dei cluster risultanti
□ Considera il contesto business
□ In caso di dubbio, preferisci K più piccolo
```

---

### Template di Codice

```python
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# Elbow + Silhouette in un loop
K_range = range(2, 11)
inertias, silhouettes = [], []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))

# K ottimale secondo Silhouette
best_k = list(K_range)[np.argmax(silhouettes)]

# Silhouette per cluster (per silhouette plot)
sil_samples = silhouette_samples(X_scaled, labels)
```

---

### Quando Usare Quale Metodo

| Situazione | Metodo Consigliato |
|------------|-------------------|
| Prima analisi esplorativa | Elbow + Silhouette |
| K candidati da confrontare | Silhouette Plot |
| Verifica statistica | Gap Statistic |
| Dati grandi (>10k punti) | Elbow (più veloce) |
| Cluster di qualità diversa | Silhouette Plot (per cluster) |

---

### Flusso Decisionale

```
        DATI SCALATI
              │
    ┌─────────┴─────────┐
    ▼                   ▼
  ELBOW            SILHOUETTE
    │                   │
    ▼                   ▼
  K₁ = gomito      K₂ = max score
    │                   │
    └─────────┬─────────┘
              │
         K₁ == K₂?
        /        \
      SI          NO
       │           │
       ▼           ▼
   Usa K₁      Silhouette Plot
                   │
              Analizza cluster
                   │
              Decidi K finale
```

---

*"Non esiste un K 'vero' - esiste il K più utile per il tuo problema.
 Usa più metodi, ma lascia che l'interpretabilità guidi la decisione finale."*